In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.layers import GRU, Input, RepeatVector, concatenate, Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from math import sqrt
from numpy import split, array
from pandas import read_csv
from keras import metrics
import os
import json
from web3 import Web3
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_log_error
import ipfshttpclient

In [13]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8599"))
print(w3.is_connected())
w3.eth.default_account=w3.eth.accounts[0]

True


In [14]:
abi=[
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "accessListAddress",
				"type": "address"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "change_train_status",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "configsCount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "deleteConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getConfiguration",
		"outputs": [
			{
				"components": [
					{
						"internalType": "uint256",
						"name": "n_past",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "n_features",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "n_future",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "lstm_units_1",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "lstm_units_2",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "dense_units_1",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "dense_units_2",
						"type": "uint256"
					},
					{
						"internalType": "bool",
						"name": "train",
						"type": "bool"
					}
				],
				"internalType": "struct Config_Data.Configuration",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "getNodeAddress",
				"type": "address"
			}
		],
		"name": "getNodeInfo",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			},
			{
				"internalType": "uint8",
				"name": "",
				"type": "uint8"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "modelConfigs",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "dense_units_2",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "train",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_configsCount",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "_train",
				"type": "bool"
			},
			{
				"internalType": "uint256",
				"name": "_n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_2",
				"type": "uint256"
			}
		],
		"name": "setConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "_train",
				"type": "bool"
			},
			{
				"internalType": "uint256",
				"name": "_n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_2",
				"type": "uint256"
			}
		],
		"name": "updateConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	}
]


In [15]:
address = "0x3F2948E1faCc1F5B394eeb0D61B83A5fF07aaEcc"
Conf_contract = w3.eth.contract(address=address, abi=abi)

In [ ]:
h=Conf_contract.functions.getConfiguration(1).call()

In [ ]:
n_past=h[0]
n_features=h[1]
n_future=h[2]
lstm_units_1=h[3]
lstm_units_2=h[4]
dense_units_1=h[5]
dense_units_2=h[6]

In [3]:
n_past=20
n_features=43
n_future=5
lstm_units_1=200
lstm_units_2=100
dense_units_1=16
dense_units_2=1

In [4]:
df1 = pd.read_csv('unit2.csv', header=0,index_col=0)
df=df1.iloc[:400000,:]

In [5]:
df=df.fillna(0)

In [6]:
df = df.iloc[:, :-4]

In [7]:
df.head()

alt      Mach        TRA          T2         T24          T30   
0  10005.0  0.448497  76.903748  502.420918  600.148034  1438.498187  \
1  10013.0  0.447741  76.903748  502.326114  600.055894  1438.350208   
2  10017.0  0.448938  77.079529  502.416067  600.210756  1439.109101   
3  10024.0  0.449883  77.079529  502.469893  600.369717  1439.240230   
4  10031.0  0.449379  77.079529  502.401271  600.298227  1439.064004   

           T48          T50        P15         P2  ...  fan_flow_mod   
0  1818.027714  1228.129848  15.806267  11.577097  ...           0.0  \
1  1817.682618  1227.879113  15.795477  11.568235  ...           0.0   
2  1820.020627  1229.422522  15.807747  11.574866  ...           0.0   
3  1819.188327  1228.538726  15.816360  11.578198  ...           0.0   
4  1818.963540  1228.389046  15.807513  11.571593  ...           0.0   

   LPC_eff_mod  LPC_flow_mod  HPC_eff_mod  HPC_flow_mod  HPT_eff_mod   
0          0.0           0.0          0.0           0.0    -0.000638  \
1          0.0           0.0          0.0           0.0    -0.000638   
2          0.0           0.0          0.0           0.0    -0.000638   
3          0.0           0.0          0.0           0.0    -0.000638   
4          0.0           0.0          0.0           0.0    -0.000638   

   HPT_flow_mod  LPT_eff_mod  LPT_flow_mod  RUL  
0           0.0          0.0           0.0   74  
1           0.0          0.0           0.0   74  
2           0.0          0.0           0.0   74  
3           0.0          0.0           0.0   74  
4           0.0          0.0           0.0   74  

[5 rows x 43 columns]

In [ ]:
train=df[:300000]
test=df[300000:]

In [8]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df.values)


In [ ]:
scaled_train = train.to_numpy()
scaled_test = test.to_numpy()
np.shape(scaled_train),np.shape(scaled_test)

In [9]:
# Prepare the input and target data
def create_dataset(data, n_past, n_future):
    X, y = [], []
    for i in range(n_past, len(data) - n_future + 1):
        X.append(data[i - n_past:i])
        y.append(data[i:i + n_future, -1])
    return np.array(X), np.array(y)



X, y = create_dataset(scaled, n_past, n_future)


In [ ]:
X_train, y_train = create_dataset(scaled_train, n_past, n_future)

In [ ]:
X_test, y_test = create_dataset(scaled_test, n_past, n_future)

In [ ]:
X_train.shape,y_train.shape

In [10]:
# Encoder
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(lstm_units_1, return_sequences=True, dropout=0.3)(encoder_inputs)
encoder_l2 = tf.keras.layers.LSTM(lstm_units_2, return_state=True, dropout=0.3)(encoder_l1)
encoder_outputs, state_h, state_c = encoder_l2
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs)
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True, dropout=0.3)(decoder_inputs, initial_state=encoder_states)

# Additional Dense layers
decoder_dense1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation='relu'))(decoder_l1)
decoder_dense2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dense_units_1, activation='relu'))(decoder_dense1)

# Final output layer
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dense_units_2))(decoder_dense2)

# Model
model_e1d1 = tf.keras.models.Model(encoder_inputs, decoder_outputs)

model_e1d1.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 43)]     0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 20, 200)      195200      ['input_1[0][0]']                
                                                                                                  
 lstm_1 (LSTM)                  [(None, 100),        120400      ['lstm[0][0]']                   
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                              

In [ ]:

# Assuming the input shapes
n_past = 20
n_features = 43
n_future = 5  # Adjust this value as per your requirements

lstm_units_1 = 100
lstm_units_2 = 100  # Change this to match the number of units in the LSTM layer of the decoder
dense_units_1 = 64
dense_units_2 = 1

# Encoder
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(lstm_units_1, return_sequences=True, dropout=0.3)(encoder_inputs)
encoder_l2 = tf.keras.layers.LSTM(lstm_units_2, return_state=True, dropout=0.3)(encoder_l1)
encoder_outputs, state_h, state_c = encoder_l2
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs)
decoder_l1 = tf.keras.layers.LSTM(lstm_units_1, return_sequences=True, dropout=0.3)(decoder_inputs, initial_state=encoder_states)

# Additional Dense layers
decoder_dense1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dense_units_1, activation='relu'))(decoder_l1)
decoder_dense2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dense_units_1, activation='relu'))(decoder_dense1)

# Final output layer
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dense_units_2))(decoder_dense2)

# Model
model_e1d2 = tf.keras.models.Model(encoder_inputs, decoder_outputs)


model_e1d2.compile(optimizer='rmsprop', loss='mse')
model_e1d2.summary()

In [ ]:


# Train model
history = model_e1d2.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Make predictions
y_pred = model_e1d2.predict(X_test)

# Reshape y_pred and y_test for error metrics calculation
y_pred = y_pred.reshape(-1)
y_test = y_test.reshape(-1)

# Calculate error metrics
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

# Print error metrics
print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAE: ', mae)

# Plot predicted vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted')
plt.show()

In [11]:
cp5 = ModelCheckpoint('model_e1d1/', save_best_only=True)
model_e1d1.compile(loss="mse", optimizer=Adam(learning_rate=0.001), metrics=[metrics.mean_squared_error,
                       metrics.mean_absolute_error,
                        tf.keras.metrics.RootMeanSquaredError(),                                                      
                       metrics.mean_absolute_percentage_error])

In [12]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_log_error

# Walk-forward validation parameters
n_splits = 3
n_train = X.shape[0] // n_splits
# Initialize performance metrics
mse_scores = []
mae_scores = []
r2_scores = []
nrmse_scores = []
rmse_scores = []
mase_scores = []
mape_scores = []
smape_scores = []

# Walk-forward validation loop
for i in range(n_splits):
    # Split data into train and test sets
    train_data = scaled[: n_train * (i + 1)]
    test_data = scaled[n_train * (i + 1): n_train * (i + 2)]

    # Create input features and target values
    X_train, y_train = create_dataset(train_data, n_past, n_future)
    X_test, y_test = create_dataset(test_data, n_past, n_future)

    # Train the model
    model_e1d1.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)

    # Generate predictions on test data
    predictions = model_e1d1.predict(X_test)

    # Flatten the 3D arrays to 2D arrays
    y_test_flat = y_test.reshape(-1, 1)
    predictions_flat = predictions.reshape(-1, 1)

    # Calculate performance metrics
    mse = mean_squared_error(y_test_flat, predictions_flat)
    mae = mean_absolute_error(y_test_flat, predictions_flat)
    r2 = r2_score(y_test_flat, predictions_flat)
    rmse = np.sqrt(mse)
    nrmse = rmse / (y_test_flat.max() - y_test_flat.min())
    mean_absolute_error_naive = np.mean(np.abs(y_test_flat[1:] - y_test_flat[:-1]))
    mase = mae / mean_absolute_error_naive
    mape = mean_absolute_percentage_error(y_test_flat, predictions_flat)
    smape = mean_absolute_percentage_error(y_test_flat, predictions_flat, multioutput='uniform_average')

    # Append the scores to the corresponding lists
    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)
    nrmse_scores.append(nrmse)
    rmse_scores.append(rmse)
    mase_scores.append(mase)
    mape_scores.append(mape)
    smape_scores.append(smape)

# Average performance metrics
avg_mse = np.mean(mse_scores)
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)
avg_nrmse = np.mean(nrmse_scores)
avg_rmse = np.mean(rmse_scores)
avg_mase = np.mean(mase_scores)
avg_mape = np.mean(mape_scores)
avg_smape = np.mean(smape_scores)

print("Average MSE:", avg_mse)
print("Average MAE:", avg_mae)
print("Average R-squared:", avg_r2)
print("Average NRMSE:", avg_nrmse)
print("Average RMSE:", avg_rmse)
print("Average MASE:", avg_mase)
print("Average MAPE:", avg_mape)
print("Average SMAPE:", avg_smape)


1/1 [==============================] - 0s 38ms/step
Average MSE: 0.01471855243534098
Average MAE: 0.07814600481867469
Average R-squared: -0.7933000167942925
Average NRMSE: inf
Average RMSE: 0.09736220320088725
Average MASE: inf
Average MAPE: 27842959576217.457
Average SMAPE: 27842959576217.457


C:\Users\montd\AppData\Local\Temp\ipykernel_17692\1970876372.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  nrmse = rmse / (y_test_flat.max() - y_test_flat.min())
C:\Users\montd\AppData\Local\Temp\ipykernel_17692\1970876372.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  mase = mae / mean_absolute_error_naive


In [ ]:
predictions = model_e1d1.predict(X_test)

In [ ]:
predictions_flat

In [ ]:
y_test_flat

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_true, y_pred, alpha=0.3)  # Plot the raw data
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted')

# Plot the line of perfect prediction
min_val = min(min(y_true), min(y_pred))
max_val = max(max(y_true), max(y_pred))
plt.plot([min_val, max_val], [min_val, max_val], 'r', lw=2)

plt.show()

In [16]:
# Save model architecture to JSON
model_json = model_e1d1.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save weights to HDF5
model_e1d1.save_weights("model.h5")

In [17]:
# Connect to IPFS
client = ipfshttpclient.connect("/ip4/127.0.0.1/tcp/5001/http")

# Add model architecture and weights to IPFS
res = client.add("model.json", "model.h5")

C:\Users\montd\AppData\Local\Programs\Python\Python310\lib\site-packages\ipfshttpclient\client\__init__.py:73: VersionMismatch: Unsupported daemon version '0.20.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [18]:
abi1=[
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "modelCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_modelData2",
				"type": "string"
			}
		],
		"name": "setModelData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "dataCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_data2",
				"type": "string"
			}
		],
		"name": "setTrainingData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "accessListAddress",
				"type": "address"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "modelCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_modelData2",
				"type": "string"
			}
		],
		"name": "updateModelData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "dataCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_data2",
				"type": "string"
			}
		],
		"name": "updateTrainingData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataMap",
		"outputs": [
			{
				"internalType": "string",
				"name": "data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "data2",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getModelData",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "modelData1",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "modelData2",
						"type": "string"
					}
				],
				"internalType": "struct Publish.ModelData",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "getNodeAddress",
				"type": "address"
			}
		],
		"name": "getNodeInfo",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			},
			{
				"internalType": "uint8",
				"name": "",
				"type": "uint8"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getTrainingData",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "data1",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "data2",
						"type": "string"
					}
				],
				"internalType": "struct Publish.TrainingData",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "modelMap",
		"outputs": [
			{
				"internalType": "string",
				"name": "modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "modelData2",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]

In [19]:
address1 = "0x6041cc1c53F338281FB623b409Ebd90e40557f71"
Pub_contract = w3.eth.contract(address=address1, abi=abi1)

In [ ]:
from web3 import Web3, IPCProvider

In [20]:
print(res[0]['Hash'])
print(res[1]['Hash'])

QmTXcazznat4DkR83GMjYauQA6kuG7XJvwXyyXrnNwxoZ8
Qmebx2pUFKEoTcTTbMcgfrGGtPBd4Ukc8Np4nY7RFmugHc


In [ ]:
Pub_contract.functions.setModelData(2,res[0]['Hash'],res[1]['Hash']).transact()

In [25]:
Pub_contract.functions.updateModelData(2,res[0]['Hash'],res[1]['Hash']).transact()

HexBytes('0x954491eae71dc4c0e7a7804341600c894d46e801398d42c58b3e1083d3cb8fc9')

In [26]:
Pub_contract.functions.updateModelData(1,res[0]['Hash'],res[1]['Hash']).transact()

HexBytes('0xf1c8b4d84916b83f5c255b8a6d69450196ddf3f589781ee45a1eee964ce8694c')

In [ ]:
# Get hashes
model_json_hash = next((file for file in res if file['Name'] == 'model.json'), None)['Hash']
model_h5_hash = next((file for file in res if file['Name'] == 'model.h5'), None)['Hash']


In [ ]:
# Retrieve files from IPFS
model_json_file = client.cat(model_json_hash)
model_h5_file = client.cat(model_h5_hash)

In [ ]:
# Save files locally
with open('retrieved_model.json', 'w') as file:
    file.write(model_json_file.decode('utf-8'))

with open('retrieved_model.h5', 'wb') as file:
    file.write(model_h5_file)

In [ ]:
import keras 
# Load model architecture from JSON
with open('retrieved_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = keras.models.model_from_json(loaded_model_json)

# Load weights into the model
loaded_model.load_weights("retrieved_model.h5")

In [ ]:
loaded_model.summary()